Run the first 2 code cells only, this would generate a dataset.csv in the current directory and add the label of sonrisa (smile) to all the frames of the video taken, you must re run this process each time you desire to add a new feature to the dataset

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mp_face_mesh = mp.solutions.face_mesh

class Facemesh:
    def __init__(self, label):
        left_eye = [263, 249, 390, 373, 374, 380, 381, 382, 362, 466, 388, 387, 386, 385, 384, 398]
        left_eyebrow = [276, 283, 282, 295, 285, 300, 293, 334, 296, 336]
        lips = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 185, 40, 39, 37, 0, 267, 269, 270, 409, 78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308, 191, 80, 81, 82, 13, 312, 311, 310, 415]
        right_eye = [33, 7, 163, 144, 145, 153, 154, 155, 133, 246, 161, 160, 159, 158, 157, 173]
        right_eyebrow = [46, 53, 52, 65, 55, 70, 63, 105, 66, 107]
        self.kps = left_eye + left_eyebrow + lips + right_eye + right_eyebrow
        mp_face_mesh = mp.solutions.face_mesh
        self.label = label
        self.face_mesh = mp_face_mesh.FaceMesh(  max_num_faces=1,
                                            refine_landmarks=True,
                                            min_detection_confidence=0.5,
                                            min_tracking_confidence=0.5)

    def get_image_landmarks(self,image):
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        return results
            
    def insert_dataframe(self, points):
        df = pd.DataFrame(points)
        print(df.to_string())
        df.to_csv('dataset.csv',mode='a')
        
                        
    def draw_image_kps(self,image,facial_landmarks):
        height, width, _ = image.shape
        points = {}
        for i in range(len(facial_landmarks.landmark)):
            if i in self.kps :
                pt= facial_landmarks.landmark[i]
                points[str(i)] = [pt.x, pt.y]
                x = int(pt.x * width)
                y = int(pt.y * height)
                cv2.circle(image, (x, y), 1, (0, 100, 255), -1)
        points['label'] = self.label
        self.insert_dataframe(points)
        cv2.imshow("Image", cv2.flip(image, 1))
        cv2.waitKey(1)

In [2]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

face_mesh = Facemesh("sonrisa")
num_frames = 0
max_frames = 200

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue
    
    if num_frames>=max_frames:
        break
    else:
        num_frames+=1
   
    multi_face_lm = face_mesh.get_image_landmarks(image)
    if multi_face_lm.multi_face_landmarks:
        for face_landmarks in multi_face_lm.multi_face_landmarks:
            face_mesh.draw_image_kps(image,face_landmarks)
cv2.destroyAllWindows()
cap.release()

          0         7        13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375      380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.575618  0.536505  0.574749  0.574663  0.573238  0.533855  0.56725

          0         7        13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.581743  0.536915  0.581140  0.581406  0.580282  0.534188  0.5727

          0         7        13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87       88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321      324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.580752  0.536845  0.580251  0.580169  0.579572  0.534146  0.571624

          0         7        13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.567892  0.516043  0.566962  0.567155  0.567178  0.513124  0.5579

          0         7      13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.535283  0.480837  0.5353  0.535299  0.536092  0.477807  0.524417  

...

          0         7        13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.579681  0.518170  0.577985  0.578159  0.577873  0.514308  0.5660

         0         7        13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.57930  0.515107  0.578785  0.578763  0.578838  0.511614  0.565665

          0         7        13        14        17        33        37        39        40        46        52        53       55        61        63        65        66        70        78        80        81       82       84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.525923  0.462252  0.526225  0.526242  0.526854  0.458922  0.512669 

          0         7        13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.460677  0.399736  0.461146  0.461160  0.461558  0.396384  0.4461

          0         7        13        14        17        33        37        39        40        46        52        53        55        61        63        65        66        70        78        80        81        82        84        87        88        91        95       105       107       133       144       145       146       153       154       155       157       158       159       160       161       163       173       178       181       185       191       246       249       263       267       269       270       276       282       283       285       291       293       295       296       300       308       310       311       312       314       317       318       321       324       334       336       362       373       374       375       380       381       382       384       385       386       387       388       390       398       402       405       409       415       466    label
0  0.436266  0.366989  0.436230  0.436326  0.436148  0.363400  0.4205

This is for getting all the points that we desire as the contour

In [5]:

left_eye = [(263, 249), (249, 390), (390, 373), (373, 374),
                               (374, 380), (380, 381), (381, 382), (382, 362),
                               (263, 466), (466, 388), (388, 387), (387, 386),
                               (386, 385), (385, 384), (384, 398), (398, 362)]

FACEMESH_LIPS = [(61, 146), (146, 91), (91, 181), (181, 84), (84, 17),
                           (17, 314), (314, 405), (405, 321), (321, 375),
                           (375, 291), (61, 185), (185, 40), (40, 39), (39, 37),
                           (37, 0), (0, 267),
                           (267, 269), (269, 270), (270, 409), (409, 291),
                           (78, 95), (95, 88), (88, 178), (178, 87), (87, 14),
                           (14, 317), (317, 402), (402, 318), (318, 324),
                           (324, 308), (78, 191), (191, 80), (80, 81), (81, 82),
                           (82, 13), (13, 312), (312, 311), (311, 310),
                           (310, 415), (415, 308)]
LEFT_EYEBROW = [(276, 283), (283, 282), (282, 295),
                                   (295, 285), (300, 293), (293, 334),
                                   (334, 296), (296, 336)]
RIGHT_EYE = [(33, 7), (7, 163), (163, 144), (144, 145),
                                (145, 153), (153, 154), (154, 155), (155, 133),
                                (33, 246), (246, 161), (161, 160), (160, 159),
                                (159, 158), (158, 157), (157, 173), (173, 133)]
RIGHT_EYEBROW = [(46, 53), (53, 52), (52, 65), (65, 55),
                                    (70, 63), (63, 105), (105, 66), (66, 107)]
a = []
for i in left_eye:
    if i[0] not in a:
        a.append(i[0])
    if  i[1] not in a:
        a.append(i[1])
print(a)

[263, 249, 390, 373, 374, 380, 381, 382, 362, 466, 388, 387, 386, 385, 384, 398]


This fragments of code should not be runned just examples and past code

In [ ]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()
 
center_ids = [468, 473]

while True:
    # Image
    
    ret, image = cap.read()
    if ret is not True:
        break
    
    height, width, _ = image.shape
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb_image)
    
    if result.multi_face_landmarks != None:

        for facial_landmarks in result.multi_face_landmarks:
            for i in range(len(facial_landmarks.landmark)):
                pt1 = facial_landmarks.landmark[i]
                x = int(pt1.x * width)
                y = int(pt1.y * height)
                cv2.circle(image, (x, y), 1, (100, 100, 0), -1)
    
    cv2.imshow("Image", image)
    cv2.waitKey(1)

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

left_eye = [263, 249, 390, 373, 374, 380, 381, 382, 466, 388, 387, 386, 385, 384, 398]

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
